If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
!apt install git-lfs

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [4]:
import transformers

print(transformers.__version__)

4.28.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [6]:
from datasets import load_dataset

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [9]:
datasets = load_dataset("text", data_files={"train": "../preprocessing/txt_files/no_action/train_output.txt", "validation": "../preprocessing/txt_files/no_action/dev_output.txt"})
# len(datasets["train"])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [10]:
datasets["train"][10]

{'text': "customer: Hi, is this jacket dry clean only? agent: Hello, it is not dry clean only. We use high quality fabric in our materials, so you can wash and dry them in a typical washer and dryer! customer: Great! It is a little inconvenient when one is not machine washable. agent: I highly agree! Especially with everything happening right now, you may not even be able to find one that is open! agent: Anyway, is there anything else I can help you with? customer: Do they come in different colors agent: Hello, I would have to direct you to our website, as our stock is very dynamic. If you do not see the color you like, I suggest checking back at the end of the month, when we refill the stock customer: That's alright. I actually want it in red if you have it. But if not, I'll get this. agent: That sounds great! Is there anything else I can help you with? customer: this is good. thank you agent: Have a great day!"}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,text
0,"agent: Hi, how can I help you? customer: Yes im looking at my account and i see i was charged for an order i never bought agent: I am so sorry about that. agent: Can I have your name? customer: Crystal Minh action: Account has been pulled up for Crystal Minh. agent: Can I also get your username, email address, and order ID? customer: crystalminh749, order id 2540170116 and my email is crystalminh749@email.com action: Purchase validation in progress ... action: Querying the system for an answer ... agent: I am very sorry about that, it was an error on our part. agent: What was the amount you were charged? customer: $49 action: A refund has been made for the amount of $49. agent: Okay I have refunded your account. agent: I am sorry about that. customer: thats okay, thank you agent: Can I help you with anything else? customer: that will be all agent: Okay, have a good day"
1,"agent: Hello, and thanks for choosing AcmeBrands! How can I help you today? customer: Hello there, I would like to check the status of my order, because I only ordered one item, however, my email confirmation says I ordered two items. agent: Okay, I'm more than happy to help you. agent: I'll need to pull up your account. Can I get your full name or account ID, please? customer: Great! Do you need any information from me? agent: Yes, so that I can pull up your account and verify everything. customer: My name is Joseph Banter and my account ID is: YKKSAJD4DY agent: Okay, thank you. action: Account has been pulled up for Joseph Banter. agent: And do you have the order ID? customer: Yes, I sure do! My order ID is: 0830865726 action: Identity verification in progess ... agent: I have successfully verified your identity! agent: I'm going to check on this error now. customer: Great! action: Querying the system for an answer ... customer: Thank you! agent: Good news, the email had an error on it. agent: You were only charged for one item. agent: Is there anything else that I can help you with? customer: Ahh, that IS good news! customer: No thank you, that is perfect! Thank you for your assistance! Have a great day! agent: You too, thanks!"
2,"customer: Hello. agent: Thank you for contacting AcmeBrands. agent: How can I help today? customer: I have some stuff in my shopping cart and I want to know if I can get it shipped for free. agent: Let me take a look for you. action: Searching the FAQ pages ... action: System Action: search pricing customer: thank you. I'm on a strict budget but i need some clothes for my foster kids. agent: If the total pre-tax cost of your order is greater than $25.00 then your cart qualifies for free shipping (5 business days). Otherwise, shipping is a flat rate of $4.99. action: FAQ answer related to pricing (question4) was selected. customer: oh it's well over 25.00 so it should be free? agent: Correct. agent: Is there anything else I can help you with today? customer: no, that's it thank agent: You're welcome, have a great day!"
3,"agent: Hi, how can I help you? customer: Hello my name is Doming Rodriguez and I want to buy a pair of jeans customer: the issue is that my card doesnt go through agent: ok, I should be able to help with that customer: ok thanks agent: So when you try to purchase the jeans on our website, you can't use your credit card? customer: yes that is correct customer: I tried using it like 5 times agent: The first thing seems obvious, but can you check that your expiration date isn't expired on the card? action: Details of troubleshoot have been entered. customer: Its not expired agent: Great, I will try to help fix the problem agent: Next, can you try logging out of our site and then logging back in. This often fixes the issue customer: Ok thanks customer: It seems like that worked agent: your welcome action: Agent is looking for solutions ... customer: I can use my card now thanks agent: Great! agent: can I help with anything else? customer: no that would be all agent: Ok, have a great day! customer: you too"

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [12]:
# can change to gpt2-small -> try a different model to train, always good to try more models
# https://huggingface.co/distilgpt2 80 mill
# https://huggingface.co/gpt2-medium -> can try this one, 360 mill
model_checkpoint = "gpt2-medium"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [13]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [15]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [16]:
tokenized_datasets["train"][1]

{'input_ids': [25781,
  25,
  922,
  6672,
  11,
  703,
  460,
  314,
  1037,
  345,
  30,
  6491,
  25,
  655,
  2227,
  284,
  2198,
  319,
  262,
  3722,
  286,
  257,
  12929,
  5797,
  25,
  1654,
  11,
  561,
  345,
  1577,
  502,
  534,
  1336,
  1438,
  393,
  1848,
  4522,
  6491,
  25,
  47319,
  28092,
  9643,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  5797,
  25,
  3224,
  284,
  428,
  345,
  561,
  1577,
  502,
  262,
  1502,
  4522,
  290,
  3053,
  5797,
  25,
  3387,
  6491,
  25,
  9225,
  1433,
  3134,
  2414,
  1983,
  6491,
  25,
  49812,
  8538,
  24,
  2670,
  31,
  12888,
  13,
  785,
  6491,
  25,
  645,
  18572,
  5797,
  25,
  2279,
  287,
  1502,
  11,
  2582,
  314,
  481,
  7603,
  262,
  3722,
  286,
  534,
  12929,
  13,
  6491,
  25,
  1049,
  6491,
  25,
  1309,
  502,
  760,
  5797,
  25,
  632,
  318,
  3058,
  287,
  4371,
  290,
  262,
  6074,
  2446,
  351,
  543,
  340,
  318,
  852,
  13686,
  318,
  2691,
  3371,
  534,
  3884,
  2657,
  13

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [17]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [18]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [19]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/8034 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1004 [00:00<?, ? examples/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [20]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

" a bronze agent: ok, was the purchase made in the last 90 days? customer: No, I bought it in November. agent: ok, unfortunately because it has been more than 90 days we cannot accept the return. Would there be anything else I can help you with? customer: What if I ask really, really nicely? agent: I can escalate to my manager if you'd like agent: I'd just need your phone number. customer: (977) 625-2661 customer: I'll look forward to hearing from them. customer: Thanks for trying to help. agent: OK, I have let my manager know"

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [21]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [22]:
from transformers import Trainer, TrainingArguments

In [23]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-noaction-ABCD",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

# if you run into memory error use
# per_device_train_batch_size (reduce to 1, or increase to 2... change etc.)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

Cloning https://huggingface.co/xrzhangli/gpt2-medium-finetuned-noaction-ABCD into local empty directory.


And we can train our model:

In [25]:
trainer.train()

/Users/graceli/miniconda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5283 [00:00<?, ?it/s]

{'loss': 2.2668, 'learning_rate': 1.8107136096914633e-05, 'epoch': 0.28}
{'loss': 2.033, 'learning_rate': 1.6214272193829265e-05, 'epoch': 0.57}
{'loss': 1.9431, 'learning_rate': 1.4321408290743897e-05, 'epoch': 0.85}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.8625001907348633, 'eval_runtime': 258.1583, 'eval_samples_per_second': 6.887, 'eval_steps_per_second': 0.864, 'epoch': 1.0}
{'loss': 1.8827, 'learning_rate': 1.2428544387658528e-05, 'epoch': 1.14}
{'loss': 1.8259, 'learning_rate': 1.0535680484573161e-05, 'epoch': 1.42}
{'loss': 1.8101, 'learning_rate': 8.642816581487791e-06, 'epoch': 1.7}
{'loss': 1.7971, 'learning_rate': 6.749952678402424e-06, 'epoch': 1.99}


...

Several commits (2) will be pushed upstream.


{'loss': 1.7464, 'learning_rate': 1.0713609691463186e-06, 'epoch': 2.84}


  0%|          | 0/223 [00:00<?, ?it/s]

{'eval_loss': 1.7848807573318481, 'eval_runtime': 221.1245, 'eval_samples_per_second': 8.041, 'eval_steps_per_second': 1.008, 'epoch': 3.0}
{'train_runtime': 56605.2556, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.093, 'train_loss': 1.8728236859661147, 'epoch': 3.0}


TrainOutput(global_step=5283, training_loss=1.8728236859661147, metrics={'train_runtime': 56605.2556, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.093, 'train_loss': 1.8728236859661147, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [23]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/223 [00:00<?, ?it/s]

Perplexity: 8.29


You can now upload the result of the training to the Hub, just execute this instruction:

In [24]:
trainer.push_to_hub()

Adding files tracked by Git LFS: ['runs/Oct17_21-25-01_Graces-MacBook-Pro-104.local/events.out.tfevents.1697598651.Graces-MBP-104.fios-router.home.75929.2']. This may take a bit of time if the files are large.
Several commits (22) will be pushed upstream.
The progress bars may be unreliable.
batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.
error: failed to push some refs to 'https://github.com/graceli458/asapp-2b'



OSError: batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.
error: failed to push some refs to 'https://github.com/graceli458/asapp-2b'


You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```